In [9]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import train_utils
%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
True


In [10]:
# read in the extracted text file
with open('datasets/text8_train') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

batches = train_utils.get_batches(encoded, 10, 50)
x, y = next(batches)

# this is what the batches look like, note that y is just x shifted back by one. 
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

 anarchism originated as a term of abuse first used against early working class radicals including t
x
 [[11  1  3  7 10 20 21 10  9  2]
 [10  9  8 10  2  1  9 14 24 10]
 [24 22 22 10 21 25 25  9 25 11]
 [22  9 24 18 25  2  9 15  2 18]
 [16  9 24 22 24  1  9 14  7  3]
 [25  9 12  0 21 16  9 25  0  2]
 [ 9 24 23 24 10 15  0 11 25  1]
 [ 9 25  2  1 21 12 11  1 21 25]
 [25 12  9 11 23  9 23  2 10 12]
 [ 9 23  2  7 23  9  2  8  9 24]]

y
 [[ 1  3  7 10 20 21 10  9  2 10]
 [ 9  8 10  2  1  9 14 24 10 12]
 [22 22 10 21 25 25  9 25 11 19]
 [ 9 24 18 25  2  9 15  2 18 18]
 [ 9 24 22 24  1  9 14  7  3 18]
 [ 9 12  0 21 16  9 25  0  2 13]
 [24 23 24 10 15  0 11 25  1  9]
 [25  2  1 21 12 11  1 21 25  9]
 [12  9 11 23  9 23  2 10 12  0]
 [23  2  7 23  9  2  8  9 24  9]]


In [14]:
print(len(text))

80000000


In [15]:
if 'net' in locals():
    del net

net = CharRNN(chars, n_hidden=512, n_layers=1)
print(net)

n_seqs, n_steps = 128, 100
train_utils.train(net, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=2000)

CharRNN(
  (lstm): LSTM(27, 512, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=27, bias=True)
)
Epoch: 1/25... Step: 1... Loss: 3.3429... Val Loss: 3.1130
Epoch: 1/25... Step: 2001... Loss: 1.5485... Val Loss: 1.5333
Epoch: 1/25... Step: 4001... Loss: 1.4159... Val Loss: 1.4144
Epoch: 2/25... Step: 6001... Loss: 1.2993... Val Loss: 1.3627
Epoch: 2/25... Step: 8001... Loss: 1.3355... Val Loss: 1.3317
Epoch: 2/25... Step: 10001... Loss: 1.2878... Val Loss: 1.3105
Epoch: 3/25... Step: 12001... Loss: 1.2880... Val Loss: 1.2949
Epoch: 3/25... Step: 14001... Loss: 1.2488... Val Loss: 1.2865
Epoch: 3/25... Step: 16001... Loss: 1.3048... Val Loss: 1.2762
Epoch: 4/25... Step: 18001... Loss: 1.2987... Val Loss: 1.2644
Epoch: 4/25... Step: 20001... Loss: 1.2651... Val Loss: 1.2624
Epoch: 4/25... Step: 22001... Loss: 1.2569... Val Loss: 1.2528
Epoch: 5/25... Step: 24001... Loss: 1.1973... Val Loss: 1.2471
Epoch: 5/25... Step:

In [ ]:
# old code used to add typos to the language generation training dataset. ignore this. 
with open('/content/drive/MyDrive/UCLA/Courses/NLP/CS 263 Final Project/text8_train') as f:
    text_train = f.read()
text_train_noised = ''
for char in text_train:
    if len(text_train_noised)%300000 ==0:
        print(len(text_train_noised)/80000000)
    with open('/content/drive/MyDrive/UCLA/Courses/NLP/CS 263 Final Project/text8_train_noised','w') as f:
        f.write(text_train_noised)
    text_train_noised = text_train_noised+net.chars[np.argmax(letter_to_emg_sim(char, net.chars, typing_style='skilled'))]